In [40]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import torch
import torch.nn as nn
import torch.optim as optim

## Test Precision @K

In [38]:
data = {
    'user':np.random.randint(10),
    'item':np.arange(10)
}

In [39]:
pprint(data)

{'item': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'user': 7}


In [123]:
df = pd.read_csv('data/real.csv',index_col=0)

In [124]:
df = df.iloc[0:,[0,1,2]]
df

,Store,User,Rating
0,NAP's Coffee & Roasters,Nuunet D.,5
1,NAP's Coffee & Roasters,vanai sun,3
2,NAP's Coffee & Roasters,Phonanan Seeon,5
3,NAP's Coffee & Roasters,Chavakorn Laophetsakunchai,5
4,NAP's Coffee & Roasters,community man,5
...,...,...,...
3348,Yuanjai Cafe,อาจณรงค์ กลมเกลียว,5
3349,Yuanjai Cafe,은하,5
3350,Yuanjai Cafe,ประจงจิต จันทป,5
3351,Yuanjai Cafe,Chatthawat Warnjai,5


In [125]:
# data = df.pivot(index='User',columns='Store',values='Rating').fillna(0)
# data = data.to_numpy()
# data

In [127]:
num_users, num_items = df.User.nunique(), df.Store.nunique()


In [132]:
df.iloc[[0, 1, 3]]

,Store,User,Rating
0,NAP's Coffee & Roasters,Nuunet D.,5
1,NAP's Coffee & Roasters,vanai sun,3
3,NAP's Coffee & Roasters,Chavakorn Laophetsakunchai,5


In [128]:
train_df = df.copy().sample(frac=0.6, random_state=1)
test_df = df.copy().drop(train_df.index)

In [130]:
# Define model parameters
num_users, num_items = df.User.nunique(), df.Store.nunique()
embedding_size = 15
learning_rate = 0.001
num_epochs = 10
batch_size = 33
relu = nn.ReLU()

# Define model architecture
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embedding(user_ids)
        item_embeds = self.item_embedding(item_ids)
        dot_product = torch.sum(user_embeds * item_embeds, dim=1)
        output = relu(dot_product)
        return output

model = MatrixFactorization(num_users, num_items, embedding_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train model
indices = np.arange(df.shape[0])

for epoch in range(num_epochs):
    np.random.shuffle(indices)

    for i in range(0, num_users, batch_size):
        batch_indices = indices[i:i+batch_size]
 
        batch_user_ids = torch.LongTensor(df.iloc[batch_indices]).unsqueeze(1)
        batch_item_ids = torch.LongTensor(df.iloc[batch_indices]).unsqueeze(1)
        
        print(batch_user_ids)
        
        break

        inputs = (batch_user_ids, batch_item_ids)
        labels = torch.FloatTensor(batch_data.flatten())

        optimizer.zero_grad()
        outputs = model(*inputs)
        
        print(batch_user_ids)
        
        print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Make recommendations for a given user
user_id = 1
user_ratings = data[user_id, :]

# Find items the user has not rated yet
unrated_items = np.where(user_ratings == 0)[0]

# Predict ratings for unrated items
item_ids = torch.LongTensor(unrated_items).unsqueeze(1)
user_ids = torch.LongTensor(np.full((len(unrated_items),), user_id)).unsqueeze(1)

ratings = model(user_ids, item_ids)
predicted_ratings = ratings.detach().numpy()
recommended_items = unrated_items[np.argsort(-predicted_ratings)]

ValueError: could not determine the shape of object type 'DataFrame'